In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.7 MB/s eta 0:00:00


In [2]:
import os, copy
import pandas as pd
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
import warnings

In [4]:
origin_df = pd.read_csv('/home/inter/chat_MBTI/make_dataset/intent_model_dataset.csv')

In [5]:
df = copy.deepcopy(origin_df)
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.dropna(inplace=True)
df.drop(df[df.duplicated()].index, axis=0, inplace=True)
df.drop(df[df['intent'] == '(표현) 부정감정 표현하기'].index, axis=0, inplace=True)
df.drop(df[df['intent'] == '(표현) 긍정감정 표현하기'].index, axis=0, inplace=True)
df.drop(df[df['intent'] == '(선언/위임하기)'].index, axis=0, inplace=True)
df.drop(df.loc[df['sentence'].str.contains('\*')].index, axis=0, inplace=True)
df = df.loc[df['sentence'].str.len() <= 100]
df.reset_index(drop=True, inplace=True)
df

,sentence,intent
0,스페인 마드리드에 여행가면 하고 싶은거 있어?,(지시) 질문하기
1,나는 키키 축구 보고 싶네,(단언) 주장하기
2,축구팀 어디 경기 말이야?,(지시) 질문하기
3,거기 레알 마드리드 홈구장이 있잖아 하하,(단언) 진술하기
4,아 그 베르나베우였나?,(지시) 질문하기
...,...,...
730244,말이라도 하는 게 낫겠지?? 오백 올려 주면 진짜 좋겠다...,(단언) 주장하기
730245,최저시급도 올랐는데 연봉도 당연히 올라야 되는 것 아냐?!,(지시) 질문하기
730246,나는 재밌어서 하루 만에 다 봤는데.,(단언) 주장하기
730247,"헐, 엄청 긴데 피곤했겠다.",(단언) 주장하기


In [6]:
# 클래스 개수
lable_num = len(df['intent'].unique())
lable_num

13

In [7]:
# str -> int 라벨변경
intet_label = list(df['intent'].unique())

label_dict = {}

for idx, intent_lab in enumerate(intet_label) :
    df.loc[df['intent'] == intent_lab, 'intent'] = idx
    label_dict[idx] = intent_lab

print(label_dict)
df

{0: '(지시) 질문하기', 1: '(단언) 주장하기', 2: '(단언) 진술하기', 3: '(지시) 충고/제안하기', 4: '(언약) 약속하기(제3자와)/(개인적 수준)', 5: '(표현) 감사하기', 6: '턴토크 사인(관습적 반응)', 7: '(지시) 명령/요구하기', 8: '(지시) 부탁하기', 9: '(단언) 반박하기', 10: '(표현) 인사하기', 11: '(표현) 사과하기', 12: '(언약) 거절하기'}


,sentence,intent
0,스페인 마드리드에 여행가면 하고 싶은거 있어?,0
1,나는 키키 축구 보고 싶네,1
2,축구팀 어디 경기 말이야?,0
3,거기 레알 마드리드 홈구장이 있잖아 하하,2
4,아 그 베르나베우였나?,0
...,...,...
730244,말이라도 하는 게 낫겠지?? 오백 올려 주면 진짜 좋겠다...,1
730245,최저시급도 올랐는데 연봉도 당연히 올라야 되는 것 아냐?!,0
730246,나는 재밌어서 하루 만에 다 봤는데.,1
730247,"헐, 엄청 긴데 피곤했겠다.",1


In [8]:
train_data = pd.DataFrame()
test_data = pd.DataFrame()

for k,v in label_dict.items():
    len_ = df.loc[df['intent'] == k,'intent'].count()
    num_ = 100000
    if len_ > num_ :
        temp = df.loc[df['intent'] == k].sample(n=num_).reset_index(drop=True)
        train_data = pd.concat([train_data,temp.loc[:((num_-1)*8)/10]],ignore_index=True)
        test_data = pd.concat([test_data,temp.loc[((num_-1)*8)/10:]],ignore_index=True)
    else :
        temp = df.loc[df['intent'] == k].reset_index(drop=True)
        train_data = pd.concat([train_data,temp.loc[:(len_*8)/10]],ignore_index=True)
        test_data = pd.concat([test_data,temp.loc[(len_*8)/10:]],ignore_index=True)


train_data = train_data.sample(frac=1).reset_index(drop=True)
test_data = test_data.sample(frac=1).reset_index(drop=True)

In [9]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [13]:
max_seq_len = max(df['sentence'].apply(lambda x: len(str(x))))
max_seq_len

100

In [48]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):

    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        # input_id는 워드임베딩을 위한 문장의 정수인코딩
        input_id = tokenizer(example, max_length=max_seq_len, padding='max_length', truncation=True)['input_ids']

        # attention_mask는 실제단어가 위치하면 1, 패딩의 위치에는 0인 시퀀스.
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count

        # token_type_id은 세그먼트 인코딩
        token_type_id = [0] * max_seq_len

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention masklength {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length{} vs {}".format(len(token_type_id), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [ ]:
tokenizer(train_data['sentence'][0], max_length=max_seq_len, padding='max_length', truncation=True)['input_ids']

In [50]:
train_X, train_y = convert_examples_to_features(train_data['sentence'], train_data['intent'],
                                              max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 279292/279292 [00:34<00:00, 8006.22it/s]


In [51]:
test_X, test_y = convert_examples_to_features(test_data['sentence'], test_data['intent'],
                                              max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 69817/69817 [00:07<00:00, 8926.72it/s]


In [52]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=lable_num, from_pt=True)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should p

In [ ]:
model.fit(train_X, train_y, epochs=4, batch_size=64, validation_split=0.2)

In [ ]:
results = model.evaluate(test_X, test_y, batch_size=64)
print("test loss, test acc: ", results)

1152/1152 [==============================] - 61s 53ms/step - loss: 0.7661 - accuracy: 0.7326
test loss, test acc:  [0.7661464810371399, 0.7325742244720459]


In [ ]:
def sentiment_predict(new_sentence):
    input_id = tokenizer.encode(new_sentence, max_length=max_seq_len, pad_to_max_length=True)

    padding_count = input_id.count(tokenizer.pad_token_id)
    attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
    token_type_id = [0] * max_seq_len

    input_ids = np.array([input_id])
    attention_masks = np.array([attention_mask])
    token_type_ids = np.array([token_type_id])

    encoded_input = [input_ids, attention_masks, token_type_ids]

    score = np.argmax(model.predict(encoded_input)[0])

    print('sentence :',new_sentence)
    print(label_dict[score])


In [ ]:
MODEL_SAVE_PATH = os.path.join("/content/Drive/MyDrive/intent_classifier/intent_classifier")
print(MODEL_SAVE_PATH)

if os.path.exists(MODEL_SAVE_PATH):
    print(f"{MODEL_SAVE_PATH} -- Folder already exists \n")
else:
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    print(f"{MODEL_SAVE_PATH} -- Folder create complete \n")

# save tokenizer, model
model.save_pretrained(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

/content/Drive/MyDrive/intent_classifier/intent_classifier
/content/Drive/MyDrive/intent_classifier/intent_classifier -- Folder already exists 



('/content/Drive/MyDrive/intent_classifier/intent_classifier/tokenizer_config.json',
 '/content/Drive/MyDrive/intent_classifier/intent_classifier/special_tokens_map.json',
 '/content/Drive/MyDrive/intent_classifier/intent_classifier/vocab.txt',
 '/content/Drive/MyDrive/intent_classifier/intent_classifier/added_tokens.json')

In [ ]:
new_sentence = input('sentence > ')
sentiment_predict(new_sentence)

sentence > 우선 저기로 가보셔요
1/1 [==============================] - 0s 42ms/step
sentence : 우선 저기로 가보셔요
(지시) 충고/제안하기
